## Dynamic Scraping with Selenium and Chrome webdriver

#### Approx execution time: 1h 30min

This is a simple notebook to work with Selenium and Chrome Webdriver in order to scrape a dynamic website. While for static websites we can use [Beautiful Soup](https://pypi.org/project/beautifulsoup4/), when we are working with dynamic ones we need Selenium. The key aspect of [Selenium](https://pypi.org/project/selenium/) is that it executes javascript when loading a page request, this allows for dynamic websites to be scraped easily. For the purpose of this exercise we will use [Chrome WebDriver](https://chromedriver.chromium.org/), a tool for automated testing of web apps. To complete our exercise we will also use [pandas](https://pypi.org/project/pandas/), the data science and data analysis library, in order to create a dataframe and export our data to our local machine.

**Tutorial sections:**
- **Start a Headless session**: Start a WebDriver session with Selenium. It will be headless since we won't open a visual browser window and our script will "drive" it across the pages we need.
- **One page simple scraping**: retrieve all the content from one page and create a dataframe
- **Scraping across pages**: Continously load new content, then create a dataframe

**Step 1: import**

In [1]:
# Will make scraping possible by importing our webdriver and relevant options. This import section can be easily
#costumized based on your needs (for example to add particular options to your browser or to catch common errors)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time

# Will allow to request, access and write urls of images and other links
import urllib.request
import requests

# Will allow us to move the data we scrape within tables and then export them to csv
import pandas as pd

Initialize Selenium and driver instance using Chrome WebDriver. To properly install ChromeDriver you can manually set the path following [this guide](https://www.kenst.com/2015/03/including-the-chromedriver-location-in-macos-system-path/) or use `pip install webdriver-manager` to manage it.

**Step 2: start Chrome webdriver session**

For the purpose of this exercise we will get some movies from [Rotten Tomatoes](https://www.rottentomatoes.com/browse/dvd-streaming-all). We are interested in creating a dataset of titles with their ranking, date of release and cover.

In [13]:
# We start an headless Chrome session. (Headless means that we don't open a visual browser window)
options = Options()
#If you comment out this option below you will be able to see Chrome opening and performing operations!
options.headless = True
options.add_argument("--window-size=1920,1200")
options.add_argument("--disable-gpu");

# instead of setting path manually (still possible but a bit annoying) we use DriverManager
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/Users/francescamorini/.wdm/drivers/chromedriver/mac64/88.0.4324.96/chromedriver] found in cache


In [14]:
driver.get("https://www.rottentomatoes.com/browse/dvd-streaming-all")

# print(driver.page_source)
# remove the comment above to print the result of our driver.get()
print('done')

done


### One time simple scraping

The concept behind scraping is quite easy to grasp. We are creating a script that behaves like a human. By using the correct methods we can fake human interactions on webpage. While usually scripts "see" different things than us, Selenium, Beautiful Soup and WebDrivers are built to see pages as we would do. Scripts are obviously faster, they don't get to go to the toilet and they do everything very enthusiastically.

**Step 3: Get movies**

Now we will select specific elements we are interested in. To do so we can instruct our script to look for them on the page. Selenium provides a series of methods to do so: we can select elements according to their class name, html tag, css selector, attribute and so on. Please go to https://www.rottentomatoes.com/browse/dvd-streaming-all, right-click on any part of the screen and then select "Inspect Element" to activate the source inspector. Here you can check the page structure and select the elements you want to obtain. In our case each movie preview is wrapper in a container with class `mb-movie`. To get all the movies we will tell Selenium to collect only elements with this specific class name.

In [4]:
# Only getting first page results
movies = driver.find_elements_by_class_name('mb-movie')
# Check how many elements we obtained. It should be 32, the number of results for the first page.
print(len(movies))

32


**Step 4: loop through movies and build a list of lists containing our data**

Now that we have a list of elements we can loop over it and replicate our pattern in a more structured way. We will ask Selenium to look for text and images of movies. Again we have a wide range of opportunities. We can use the `.text` method to obtain the content of an HTML tag or we can get attributes from links, it really depends on the website structure and the information we are interested in.

In [5]:
# This empty list will later contain my precious data
moviesData = []
# I am creating a counter, this will print the parser status and generate filenames for images
count = 0
for movie in movies:
    
    count = count + 1
    
    # we can get precise elements by using css selectors, classes, tag names and xpaths
    title = movie.find_element_by_class_name('movieTitle').text
    rating = movie.find_element_by_class_name('tMeterIcon').text
    available = movie.find_element_by_class_name('release-date').text
    image = movie.find_element_by_tag_name('img')
    print(count, '/', title)
    
    # it is also possible to get accessory information from collected elements, 
    # in this case we want to get the url of the poster images, we can retrieve it from the src attribute
    source = image.get_attribute("src")
    
    # We use urllib to read the poster images and save them locally in a special folder. In order to avoid
    # overwriting we use the {0}.format('dynamic variable') to save all of them
    imgdestination = "data/img_basic/poster-{0}.jpg".format(count)
    urllib.request.urlretrieve(source, imgdestination)
    
    # We append a shorter list to the initial list
    moviesData.append([title, rating, available, imgdestination])

# Safety print to check everything is fine!
print(moviesData)

1 / Promising Young Woman
2 / Yellow Rose
3 / American Skin
4 / Sing Me A Song
5 / The Changin' Times Of Ike White
6 / Born To Be
7 / Savage
8 / The Delivered (Fanny Lye Deliver'D)
9 / Pieces Of A Woman
10 / Assassins
11 / Love Sarah
12 / One Night In Miami
13 / Notturno
14 / MLK/FBI
15 / Shadow In The Cloud
16 / Ten Minutes To Midnight
17 / I Am Lisa
18 / PG: Psycho Goreman
19 / Baby, Done
20 / We Can Be Heroes
21 / Don'T Tell A Soul
22 / Redemption Day
23 / Yearly Departed
24 / The One You Feed
25 / Stallone: Frank, That Is
26 / Stars Fell On Alabama
27 / Gun And A Hotel Bible
28 / If Not Now, When?
29 / Caged
30 / Finding 'Ohana
31 / Locked Down
32 / The Dig
[['Promising Young Woman', ' 91%', 'Available Jan 15', 'data/img_basic/poster-1.jpg'], ['Yellow Rose', ' 87%', 'Available Jan 5', 'data/img_basic/poster-2.jpg'], ['American Skin', ' 29%', 'Available Jan 15', 'data/img_basic/poster-3.jpg'], ['Sing Me A Song', ' 89%', 'Available Jan 1', 'data/img_basic/poster-4.jpg'], ["The Changi

**Step 5: Move list of lists content to a pandas dataframe**

We use pandas to create a dataframe holding the information we gathered from Rotten Tomatoes. This is the gateway to data analysis and data cleaning. From pandas we can export a `.csv` file that can later be cleaned, loaded in various applications and finally used to code our prototype.

In [6]:
AllStreamingTitles = pd.DataFrame(moviesData, columns=["title", "rating", "available", "img"])

In [7]:
#Previewing 5 random rows from my dataframe
AllStreamingTitles.sample(5)

,title,rating,available,img
15,Ten Minutes To Midnight,82%,Available Jan 19,data/img_basic/poster-16.jpg
21,Redemption Day,15%,Available Jan 12,data/img_basic/poster-22.jpg
9,Assassins,97%,Available Jan 15,data/img_basic/poster-10.jpg
22,Yearly Departed,80%,Available Dec 30,data/img_basic/poster-23.jpg
11,One Night In Miami,98%,Available Jan 15,data/img_basic/poster-12.jpg


**Step 6: Export dataframe to csv**

In [8]:
# Check the 'data' subfolder after executing this cell to see the file.
AllStreamingTitles.to_csv('data/titlesDataRaw.csv', sep=',')

### Iterating on page's elements

What if we need to scrape more than what is visible on the first page? Currently we are getting titles only from the initial screen. 
Instead we want to keep going: our script should be able to proceed down the page and do something everytime new titles need to be loaded. To approach this problem we need to go back to the source website. At the bottom of the page it's possible to spot a yellow "Show More" button, if we click on it new movies are loaded. 

**Positive aspects**
- We have a distinct element, the yellow button with fairly easy interaction
- We have a stable pattern and everything is happening on one single page, we don't need to touch the page URL
- We can adopt a similar procedure to the previous one.

**Negative aspects**
- The page indefinitely loads new content, the button does not change and/or disappear at some point

This calls for a fairly simple strategy. We will modify only one step of our procedure: step 3. Instead of creating a variable with the first page results, we will create a while loop that keeps loading content up to a certain point, while getting all the movies. This particular case does not require that we access individual URLs, for an example of cross-page scraping see the Advanced Dynamic Scraper notebook.

**Step 1 and 2 stay the same!**

**Step 3: automate clicking of the "Show More" button**

Websites generally don't like to be scraped (on some websites it's explicitly forbidden, so be careful). It can happen that if we are interacting with the page too quickly we might get blocked or throttled (slowed down). We will fake interactions with the "Show More" button every 5 seconds to avoid throttle.

In [15]:
# I am creating a variable holding the number of times I want to click on the 'Show More' button. 
# Alternatively I could code my script to stop when a certain point in data is reached, 
#for example a particular title
pageExpandLimit = 0
# I am also creating a variable to store movie pictures correctly
count = 0
# An empty list to store my webelements
moreMovies = []
# And an empty list that will be filled with my data.
dataList = []

while True:
    # Add 1 to counter so we can artificially stop the loop
    print('I clicked "show more" {0} times'.format(pageExpandLimit))
    pageExpandLimit = pageExpandLimit + 1 
    
    # Taking a screenshot everytime we click on the button (to check where my browser's at)
    # This can be remove if it ends up taking too much space.
    driver.save_screenshot("screenshots/screenshot{0}.png".format(pageExpandLimit))
    
    if pageExpandLimit < 10:
        
        # Generic asleep time to avoid throttle
        time.sleep(5)
        # Find the button element and click on it to keep loading movies
        driver.find_element_by_class_name('mb-load-btn').click()  
        # Dump all the elements in one list
        moreMovies = driver.find_elements_by_class_name('mb-movie')
        
    # It will set the condition to False if counter is greater than 10, this will break the while loop
    if pageExpandLimit > 10:
        
        # Before breaking the loop I will go through the data as I did in the example above
        print('started movies collection')
        for movie in moreMovies:
            count = count + 1
            try:
                title = movie.find_element_by_class_name('movieTitle').text
                rating = movie.find_element_by_class_name('tMeterIcon').text
                available = movie.find_element_by_class_name('release-date').text
                image = movie.find_element_by_tag_name('img')
            
                print(count, '/', title)

                source = image.get_attribute("src")

                imgdestination = "data/img_basic/poster-{0}.jpg".format(count)
                urllib.request.urlretrieve(source, imgdestination)

                # Appending data
                dataList.append([title, rating, available, imgdestination])
            except:
                print('an exception occurred for', count)
            
        # Breaking and exiting the loop
        break

I clicked "show more" 0 times
I clicked "show more" 1 times
I clicked "show more" 2 times
I clicked "show more" 3 times
I clicked "show more" 4 times
I clicked "show more" 5 times
I clicked "show more" 6 times
I clicked "show more" 7 times
I clicked "show more" 8 times
I clicked "show more" 9 times
I clicked "show more" 10 times
started movies collection
1 / Promising Young Woman
2 / Yellow Rose
3 / American Skin
4 / Sing Me A Song
5 / The Changin' Times Of Ike White
6 / Born To Be
7 / Savage
8 / The Delivered (Fanny Lye Deliver'D)
9 / Pieces Of A Woman
10 / Assassins
11 / Love Sarah
12 / One Night In Miami
13 / Notturno
14 / MLK/FBI
15 / Shadow In The Cloud
16 / Ten Minutes To Midnight
17 / I Am Lisa
18 / PG: Psycho Goreman
19 / Baby, Done
20 / We Can Be Heroes
21 / Don'T Tell A Soul
22 / Redemption Day
23 / Yearly Departed
24 / The One You Feed
25 / Stallone: Frank, That Is
26 / Stars Fell On Alabama
27 / Gun And A Hotel Bible
28 / If Not Now, When?
29 / Caged
30 / Finding 'Ohana
31 

In [18]:
print(dataList)

[['Promising Young Woman', ' 91%', 'Available Jan 15', 'data/img_basic/poster-1.jpg'], ['Yellow Rose', ' 87%', 'Available Jan 5', 'data/img_basic/poster-2.jpg'], ['American Skin', ' 29%', 'Available Jan 15', 'data/img_basic/poster-3.jpg'], ['Sing Me A Song', ' 89%', 'Available Jan 1', 'data/img_basic/poster-4.jpg'], ["The Changin' Times Of Ike White", ' 100%', 'Available Jan 26', 'data/img_basic/poster-5.jpg'], ['Born To Be', ' 100%', 'Available Jan 19', 'data/img_basic/poster-6.jpg'], ['Savage', ' 75%', 'Available Jan 29', 'data/img_basic/poster-7.jpg'], ["The Delivered (Fanny Lye Deliver'D)", ' 93%', 'Available Jan 15', 'data/img_basic/poster-8.jpg'], ['Pieces Of A Woman', ' 76%', 'Available Jan 7', 'data/img_basic/poster-9.jpg'], ['Assassins', ' 97%', 'Available Jan 15', 'data/img_basic/poster-10.jpg'], ['Love Sarah', ' 56%', 'Available Jan 15', 'data/img_basic/poster-11.jpg'], ['One Night In Miami', ' 98%', 'Available Jan 15', 'data/img_basic/poster-12.jpg'], ['Notturno', ' 80%', '

In [19]:
MoreStreamingTitles = pd.DataFrame(dataList, columns=["title", "rating", "available", "img"])

In [20]:
MoreStreamingTitles.sample(5)

,title,rating,available,img
84,Chick Fight,37%,Available Nov 13,data/img_basic/poster-85.jpg
175,Misbehaviour,86%,Available Sep 25,data/img_basic/poster-176.jpg
252,Sometimes Always Never (Triple Word Score),82%,Available Aug 18,data/img_basic/poster-253.jpg
101,J.R. 'Bob' Dobbs & The Church Of The SubGenius,100%,Available Oct 20,data/img_basic/poster-102.jpg
121,Tremors: Shrieker Island,38%,Available Oct 20,data/img_basic/poster-122.jpg


In [21]:
MoreStreamingTitles.to_csv('data/titlesDataRaw_extended.csv', sep=',')

**Steps 4, 5 and 6 are identical to the "one page simple scraping" example**

In [23]:
#Ultimately, once we got everything we were interested in, we need to quit our WebDriver to avoid sessions piling up
driver.quit()

### Conclusion

Each website is different, therefore scraping is often a tailored activity that requires some coding and strategizing effort. This tutorial presented some simple features and a couple of useful patterns if you need to scrape a relatively contained amount of data. There are more complex and articulated tasks that require more savviness, time and computing effort. One example are social media. For special or recurring cases there's a useful collection of tools that can be found [here](https://wiki.digitalmethods.net/Dmi/ToolDatabase).

### References

1. Guide to Selenium basic functions for scraping: https://www.scrapingbee.com/blog/selenium-python/
2. Including WebDriver location in MacOS System Path: https://www.kenst.com/2015/03/including-the-chromedriver-location-in-macos-system-path/
3. Data Science Skills Set: https://francesco-ai.medium.com/data-science-skills-list-9f38863adab5
4. DMI Tools Database: https://wiki.digitalmethods.net/Dmi/ToolDatabase